In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('./Data/titanic.csv')
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [9]:
data.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [10]:
#data.drop(['ticket', 'PassengerId'], axis=1, inplace=True)
data.drop(['ticket'], axis=1, inplace=True)

gender_mapper = {'male': 0, 'female': 1}
data['sex'].replace(gender_mapper, inplace=True)

data['title'] = data['name'].apply(lambda x: x.split(',')[1].strip().split(' ')[0])
data['title'] = [0 if x in ['Mr.', 'Miss.', 'Mrs.'] else 1 for x in data['title']]
data = data.rename(columns={'title': 'Title_Unusual'})
data.drop('name', axis=1, inplace=True)

data['Cabin_Known'] = [0 if str(x) == 'nan' else 1 for x in data['cabin']]
data.drop('cabin', axis=1, inplace=True)

emb_dummies = pd.get_dummies(data['embarked'], drop_first=True, prefix='embarked')
data = pd.concat([data, emb_dummies], axis=1)
data.drop('embarked', axis=1, inplace=True)

data['age'] = data['age'].fillna(int(data['age'].mean()))

In [20]:
data= data.select_dtypes(exclude='object')
data.dropna(inplace=True)

In [21]:
X = data.drop('survived', axis=1)
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [22]:
X_train.head()


,pclass,sex,age,sibsp,parch,fare,body,Title_Unusual,Cabin_Known,embarked_Q,embarked_S
505,2,0,48.0,0,0,13.0000,271.0,0,0,0,1
367,2,0,52.0,0,0,13.5000,130.0,0,0,0,1
421,2,0,24.0,0,0,13.5000,297.0,0,0,0,1
891,3,0,33.0,0,0,8.6542,285.0,0,0,0,1
919,3,0,18.5,0,0,7.2292,58.0,0,0,0,0


In [23]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [33]:
from tpot import TPOTClassifier
###Le processus de formation
#tpot = TPOTClassifier(verbosity=2, max_time_mins=10)
n_gen =2
pop_size=2
tpot = TPOTClassifier(generations=n_gen, population_size=pop_size, verbosity=2, random_state=42)

tpot.fit(X_train_scaled, y_train)

Optimization Progress:   0%|          | 0/6 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Best pipeline: GaussianNB(input_matrix)


TPOTClassifier(generations=2, population_size=2, random_state=42, verbosity=2)

In [25]:
#  exemple 2
#tpot = TPOTClassifier(generations=10, population_size=20, verbosity=2)

In [26]:
# exemple 3
#tpot = TPOTClassifier(generations=10, population_size=20, verbosity=2, scoring='f1')

In [31]:
dir(tpot)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_operators',
 '_add_terminals',
 '_check_dataset',
 '_check_feature_names',
 '_check_n_features',
 '_check_periodic_pipeline',
 '_cleanup_memory',
 '_combine_individual_stats',
 '_compile_to_sklearn',
 '_create_periodic_checkpoint_folder',
 '_evaluate_individuals',
 '_fit_init',
 '_gen_grow_safe',
 '_generate',
 '_get_make_pipeline_func',
 '_get_param_names',
 '_get_tags',
 '_import_hash',
 '_import_hash_and_add_terminals',
 '_impute_values',
 '_init_pretest',
 '_mate_operator',
 '_more_tags',
 '_operator_count',
 '_preprocess_individuals',
 '_random_mutation_operator',
 '_read_confi

In [34]:
tpot.fitted_pipeline_

Pipeline(steps=[('gaussiannb', GaussianNB())])

In [35]:
tpot.score(X_test_scaled, y_test)

/home/hadoop/.local/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


1.0